# Импортируем библиотеки



In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score, precision_score
from tqdm import tqdm
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# EDA

In [ ]:
train_data = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Colab/train.csv'))
test_data = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Colab/test.csv'))

In [ ]:
train_data.head()


,Class Index,Title,Description
0,3,Wall St. Bears Claw Back Into the Black (Reuters),"Reuters - Short-sellers, Wall Street's dwindli..."
1,3,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
2,3,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
3,3,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
4,3,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."


In [ ]:
test_data.head()

,ID,Title,Description
0,0,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,1,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,2,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,3,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [ ]:
train_data['Class Index'].value_counts()

3    30000
4    30000
2    30000
1    30000
Name: Class Index, dtype: int64

Классы сбалансированы, можно обучать на данной выборке

In [ ]:
nlp = spacy.load('en_core_web_sm')
nlp.pipe_names


['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [ ]:
a = train_data['Description'].values[0]
print(a.split())
print(a.split()[5])
b = a.split()[5]
print(b.split('\\'))

['Reuters', '-', 'Short-sellers,', 'Wall', "Street's", 'dwindling\\band', 'of', 'ultra-cynics,', 'are', 'seeing', 'green', 'again.']
dwindling\band
['dwindling', 'band']


In [ ]:
doc = nlp(a)
token = [w.text for w in doc ]
print(token)


['Reuters', '-', 'Short', '-', 'sellers', ',', 'Wall', 'Street', "'s", 'dwindling\\band', 'of', 'ultra', '-', 'cynics', ',', 'are', 'seeing', 'green', 'again', '.']


In [ ]:
print("is_alpha:", [token.is_alpha for token in doc])
print("is_punct:", [token.is_punct for token in doc])
print("like_num:", [token.like_num for token in doc])

is_alpha: [True, False, True, False, True, False, True, True, False, False, True, True, False, True, False, True, True, True, True, False]
is_punct: [False, True, False, True, False, True, False, False, False, False, False, False, True, False, True, False, False, False, False, True]
like_num: [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [ ]:
for token in doc:
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    token_head = token.head.text
    print(f"{token_text:<10}{token_pos:<10}" \
          f"{token_dep:<10}{token_head:<10}")

Reuters   PROPN     compound  sellers   
-         PUNCT     punct     sellers   
Short     PROPN     compound  sellers   
-         PUNCT     punct     sellers   
sellers   NOUN      nsubj     seeing    
,         PUNCT     punct     sellers   
Wall      PROPN     compound  Street    
Street    PROPN     poss      dwindling\band
's        PART      case      Street    
dwindling\bandNUM       appos     sellers   
of        ADP       prep      dwindling\band
ultra     NOUN      pobj      of        
-         NOUN      pobj      of        
cynics    NOUN      pobj      of        
,         PUNCT     punct     seeing    
are       AUX       aux       seeing    
seeing    VERB      ROOT      seeing    
green     ADJ       dobj      seeing    
again     ADV       advmod    seeing    
.         PUNCT     punct     seeing    


In [ ]:
from spacy import displacy
displacy.render(doc, style='dep', jupyter=True)

In [ ]:
print('Quotes:',spacy.lang.punctuation.LIST_QUOTES)
print('\nPunctuations:',spacy.lang.punctuation.LIST_PUNCT)

Quotes: ["\\'", '"', '”', '“', '`', '‘', '´', '’', '‚', ',', '„', '»', '«', '「', '」', '『', '』', '（', '）', '〔', '〕', '【', '】', '《', '》', '〈', '〉', '〈', '〉', '', '⟦', '⟧']

Punctuations: ['…', '……', ',', ':', ';', '\\!', '\\?', '¿', '؟', '¡', '\\(', '\\)', '\\[', '\\]', '\\{', '\\}', '<', '>', '_', '#', '\\*', '&', '。', '？', '！', '，', '、', '；', '：', '～', '·', '।', '،', '۔', '؛', '٪']


In [ ]:
stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
len(stopwords)



326

In [ ]:
digit = [w.text for w in doc if w.is_digit]
print('Digit in sentence: ',digit)

Digit in sentence:  []


In [ ]:
lemma = [w.lemma_ for w in doc]
print(lemma)

['Reuters', '-', 'Short', '-', 'seller', ',', 'Wall', 'Street', "'s", 'dwindling\\band', 'of', 'ultra', '-', 'cynic', ',', 'be', 'see', 'green', 'again', '.']


Преобразуем наш датасет:
*   удалим стоп-слова
*   удалим знаки пунктуации
*   удалим цифры
*   приведем к одному регистру
*   применим лемматизацию



In [ ]:
def clean_title(df):
    nlp = spacy.load('en_core_web_sm')
    for i in tqdm(range(df.shape[0])):
        doc = nlp(df['Title'][i])
        text = [w.lemma_.lower().strip() for w in doc
               if not (w.is_stop |
                    w.is_punct |
                    w.is_digit)
               ]
        text = " ".join(text)
        df['Title'][i] = text

    return df

In [ ]:
def clean_description(df):
    nlp = spacy.load('en_core_web_sm')
    for i in tqdm(range(df.shape[0])):
        doc = nlp(df['Description'][i])
        text = [w.lemma_.lower().strip() for w in doc
               if not (w.is_stop |
                    w.is_punct |
                    w.is_digit)
               ]
        text = " ".join(text)
        df['Description'][i] = text

    return df

In [ ]:
#data_processed = clean_description(train_data)

Поскольку процесс занимает время, то сделаем один раз, сохраним и загрузим уже обработанный датасет

In [ ]:
train_df = pd.DataFrame(pd.read_csv('/content/drive/MyDrive/Colab/train_lem_low.csv', sep=';', comment='#'))
train_df.drop(columns = ['Unnamed: 0'],axis = 1, inplace=True)

In [ ]:
train_df['Class Index'] = train_df['Class Index'] - 1
train_df['Description'] = train_df['Title'] + ' sep ' +  train_df['Description']

# Обучим несколько моделей

## Векторизуем с помощью CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
cv = CountVectorizer(ngram_range=(1,2), min_df = 10)
X_cv = cv.fit_transform(train_df['Description'].apply(lambda x: np.str_(x)))

In [ ]:
X_cv.shape

(120000, 47095)

In [ ]:
y = train_df['Class Index']
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=100)

### Наивный Байес Бернулли

In [ ]:
nb = BernoulliNB(alpha = 0.2)
nb.fit(x_train,y_train)

BernoulliNB(alpha=0.2)

In [ ]:
pred = nb.predict(x_test)
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print(f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Classification_report\n',classification_report(y_test,pred))

Confusion matrix
 [[8079  330  363  248]
 [  92 8783   32   52]
 [ 359  104 7727  843]
 [ 321   79  547 8041]]
0.9063888888888889
Classification_report
               precision    recall  f1-score   support

           1       0.91      0.90      0.90      9020
           2       0.94      0.98      0.96      8959
           3       0.89      0.86      0.87      9033
           4       0.88      0.89      0.88      8988

    accuracy                           0.91     36000
   macro avg       0.91      0.91      0.91     36000
weighted avg       0.91      0.91      0.91     36000



### Мультиноминальный Наивный Байес

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=0.14)
mnb.fit(x_train,y_train)

MultinomialNB(alpha=0.14)

In [ ]:
pred = mnb.predict(x_test)
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('f1_score\n', f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Classification_report\n',classification_report(y_test,pred))

Confusion matrix
 [[8086  313  386  235]
 [  89 8782   35   53]
 [ 357   90 7816  770]
 [ 332   71  609 7976]]
f1_score
 0.9072222222222223
Classification_report
               precision    recall  f1-score   support

           1       0.91      0.90      0.90      9020
           2       0.95      0.98      0.96      8959
           3       0.88      0.87      0.87      9033
           4       0.88      0.89      0.89      8988

    accuracy                           0.91     36000
   macro avg       0.91      0.91      0.91     36000
weighted avg       0.91      0.91      0.91     36000



In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'alpha':[0.8, 0.9, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.2]}

In [ ]:
mnb = MultinomialNB()

In [ ]:
grid_search = GridSearchCV(estimator=mnb, param_grid=params, scoring='f1_micro', verbose=1)

In [ ]:
grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 11 candidates, totalling 55 fits


GridSearchCV(estimator=MultinomialNB(),
             param_grid={'alpha': [0.8, 0.9, 0.1, 0.11, 0.12, 0.13, 0.14, 0.15,
                                   0.16, 0.17, 0.2]},
             scoring='f1_micro', verbose=1)

In [ ]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)

MultinomialNB(alpha=0.1)
0.9092738095238095


In [ ]:
nb = BernoulliNB()
grid_search = GridSearchCV(estimator=nb, param_grid=params, scoring='f1_micro', verbose=1)
grid_search.fit(x_train, y_train)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 5 folds for each of 11 candidates, totalling 55 fits
BernoulliNB(alpha=0.14)
0.9075


### Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

KeyboardInterrupt: ignored

In [ ]:
pred = rf.predict(x_test)
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('f1_score\n',f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Classification_report\n',classification_report(y_test,pred))

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(x_train, y_train)


KNeighborsClassifier(n_jobs=-1)

In [ ]:
pred = knn.predict(x_test)
print('f1_score\n', f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('Classification_report\n',classification_report(y_test,pred))

f1_score
 0.36877777777777776
Confusion matrix
 [[5725 3224   41   30]
 [3702 5221    2   34]
 [3678 3965 1300   90]
 [3764 4104   90 1030]]
Classification_report
               precision    recall  f1-score   support

           1       0.34      0.63      0.44      9020
           2       0.32      0.58      0.41      8959
           3       0.91      0.14      0.25      9033
           4       0.87      0.11      0.20      8988

    accuracy                           0.37     36000
   macro avg       0.61      0.37      0.33     36000
weighted avg       0.61      0.37      0.33     36000



### Дерево решений

In [ ]:
from sklearn.metrics._plot.det_curve import DetCurveDisplay
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier()

DTC.fit(x_train, y_train)

DecisionTreeClassifier()

In [ ]:
pred = DTC.predict(x_test)
print(f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('Classification_report\n',classification_report(y_test,pred))

0.8158055555555556
Confusion matrix
 [[7396  578  513  533]
 [ 377 8095  214  273]
 [ 602  294 7016 1121]
 [ 578  381 1167 6862]]
Classification_report
               precision    recall  f1-score   support

           1       0.83      0.82      0.82      9020
           2       0.87      0.90      0.88      8959
           3       0.79      0.78      0.78      9033
           4       0.78      0.76      0.77      8988

    accuracy                           0.82     36000
   macro avg       0.82      0.82      0.82     36000
weighted avg       0.81      0.82      0.82     36000



## Векторизуем с помощью TD-IDF

In [ ]:
tf = TfidfVectorizer(use_idf=True, ngram_range=(1,2))
X = tf.fit_transform(train_df['Description'].apply(lambda x: np.str_(x)))

In [ ]:
y = train_data['Class Index']
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=100)

### Наивный Байес Бернулли

In [ ]:
nb = BernoulliNB()
nb.fit(x_train,y_train)

BernoulliNB()

In [ ]:
pred = nb.predict(x_test)
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print(f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Classification_report\n',classification_report(y_test,pred))

Confusion matrix
 [[8073  342  359  246]
 [  77 8796   33   53]
 [ 381  122 7652  878]
 [ 340  100  536 8012]]
0.9036944444444446
Classification_report
               precision    recall  f1-score   support

           1       0.91      0.90      0.90      9020
           2       0.94      0.98      0.96      8959
           3       0.89      0.85      0.87      9033
           4       0.87      0.89      0.88      8988

    accuracy                           0.90     36000
   macro avg       0.90      0.90      0.90     36000
weighted avg       0.90      0.90      0.90     36000



### Мультиноминальный Наивный Байес

In [ ]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB(alpha=0.1)
mnb.fit(x_train,y_train)

MultinomialNB(alpha=0.1)

In [ ]:
pred = mnb.predict(x_test)
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('f1_score\n', f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Classification_report\n',classification_report(y_test,pred))

Confusion matrix
 [[8067  310  406  237]
 [  91 8772   42   54]
 [ 313   83 7898  739]
 [ 324   64  651 7949]]
f1_score
 0.9079444444444444
Classification_report
               precision    recall  f1-score   support

           1       0.92      0.89      0.91      9020
           2       0.95      0.98      0.96      8959
           3       0.88      0.87      0.88      9033
           4       0.89      0.88      0.88      8988

    accuracy                           0.91     36000
   macro avg       0.91      0.91      0.91     36000
weighted avg       0.91      0.91      0.91     36000



### Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

KeyboardInterrupt: ignored

In [ ]:
pred = rf.predict(x_test)
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('f1_score\n',f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Classification_report\n',classification_report(y_test,pred))

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn.fit(x_train, y_train)


KNeighborsClassifier(n_jobs=-1)

In [ ]:
pred = knn.predict(x_test)
print('f1_score\n', f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('Classification_report\n',classification_report(y_test,pred))

f1_score
 0.36877777777777776
Confusion matrix
 [[5725 3224   41   30]
 [3702 5221    2   34]
 [3678 3965 1300   90]
 [3764 4104   90 1030]]
Classification_report
               precision    recall  f1-score   support

           1       0.34      0.63      0.44      9020
           2       0.32      0.58      0.41      8959
           3       0.91      0.14      0.25      9033
           4       0.87      0.11      0.20      8988

    accuracy                           0.37     36000
   macro avg       0.61      0.37      0.33     36000
weighted avg       0.61      0.37      0.33     36000



### Дерево решений

In [ ]:
from sklearn.metrics._plot.det_curve import DetCurveDisplay
from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier()

DTC.fit(x_train, y_train)

In [ ]:
pred = DTC.predict(x_test)
print(f1_score(y_test, pred, average='micro'))
#print(accuracy_score(pred, y_test))
print('Confusion matrix\n',confusion_matrix(y_test,pred))
print('Classification_report\n',classification_report(y_test,pred))

## Векторизуем с помощью Word2vec

In [ ]:
from gensim.models import word2vec, FastText

#RNN

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix

class NamesDataset(Dataset):
    def __init__(self, X, y):
      super(NamesDataset, self).__init__()
      Xcoo = X.tocoo()
      X = torch.sparse.LongTensor(torch.LongTensor([Xcoo.row.tolist(), Xcoo.col.tolist()]),
                              torch.LongTensor(Xcoo.data.astype(np.int32)))
      self.X = X
      self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
      return self.X.shape[0]

    def __getitem__(self, index):
      return self.y[index], self.X[index]

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm_notebook


class AverageMetter:
    def __init__(self, init_val=0):
        self.value = init_val
        self.count = 0

    def update(self, new_val):
        self.value += new_val
        self.count += 1

    @property
    def avg(self):
        return self.value / self.count

    def drop(self):
        self.value = 0
        self.count = 0


def train_step(model, input_tensor, annot, loss, optimizer, device):
    hidden = model.init_hidden(input_tensor.shape[0], device)
    model.zero_grad()


    out, hidden = model(input_tensor, hidden)
    loss = loss(out, annot)
    loss.backward()
    optimizer.step()
    return loss

def train(model, dataset, loss_fn, opt, scheduler, batch_size, shuffle, num_workers, n_epochs, device):
    writer = SummaryWriter()
    ds_len = len(dataset)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, pin_memory=False,
                            drop_last=True)
    global_step_counter = 0
    loss_handler = AverageMetter()
    for epoch_id in range(n_epochs):
        loss_handler.drop()
        tq = tqdm_notebook(total=ds_len)
        tq.set_description_str(f'epoch id: {epoch_id}')
        for batch_id, batch in enumerate(dataloader):
            ann_tensor, inp_tensor = batch
            ann_tensor, inp_tensor = ann_tensor.to(device), inp_tensor.to(device)
            step_loss = train_step(model, inp_tensor, ann_tensor, loss_fn, opt, device)
            loss_handler.update(step_loss.item())
            #if batch_id % print_step == 0:
            #    print()
            writer.add_scalar('loss', step_loss, global_step_counter * batch_size)
            global_step_counter += 1
            tq.set_postfix_str(f'current iter: {global_step_counter} current loss {step_loss} avg: {loss_handler.avg}')
            tq.update(batch_size)
        scheduler.step()

In [ ]:
import torch
from torch import nn
from torch import optim

class RNN(nn.Module):
    def __init__(self, input_size, output_size, hidden_size):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size

        self.input2hidden = nn.Linear(input_size + hidden_size, hidden_size)
        self.input2output = nn.Linear(input_size + hidden_size, output_size)
        self.activation = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):

        concat = torch.cat((x, hidden), 1)
        hidden = self.input2hidden(concat)
        output = self.input2output(concat)
        output = self.activation(output)
        return output, hidden

    def init_hidden(self, batch_size, device):
        return torch.zeros(batch_size, self.hidden_size).to_sparse().to(device)


In [ ]:
47095 + 64

47159

In [ ]:
X_cv.shape

(120000, 47095)

In [ ]:
X = X_cv[:10000]
y = train_df['Class Index'].to_numpy()
dataset = NamesDataset(X, y)

criterion = nn.NLLLoss()
learning_rate = 1e-2
batch_size = 32
shuffle = True
num_workers = 0
n_epochs = 5

HIDDEN_SIZE = 64

test_rnn = RNN(X.shape[1], 4, HIDDEN_SIZE)
test_rnn.to(device)

optimizer = optim.SGD(test_rnn.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 10, 20], gamma=0.5)

train(test_rnn, dataset, criterion, optimizer, scheduler, batch_size, shuffle, num_workers, n_epochs, device)

<ipython-input-25-a7eb9713fce7>:43: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tq = tqdm_notebook(total=ds_len)


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
X = X_cv[:20000]
y = train_df['Class Index'].to_numpy()
dataset = NamesDataset(X, y)
y_true = list()
y_pred = list()
for sample_id, sample in enumerate(dataset):
    with torch.no_grad():
        hidden = test_rnn.init_hidden(1, device)
        ann_tensor, inp_tensor = sample
        ann_tensor, inp_tensor = ann_tensor.to(device), inp_tensor.to(device)
        out, _ = test_rnn(inp_tensor[None], hidden)
        y_true.append(ann_tensor.item())
        y_pred.append(out.topk(1)[1].item())
        # print(dataset.id2class(ann_tensor.item()), dataset.output2categ(out))

In [ ]:
from sklearn.metrics import f1_score

print(f1_score(y_true, y_pred, average='macro'))

0.819659568649579


In [ ]:
test_prepared = clean_description(test_data)
test_prepared = clean_title(test_prepared)

In [ ]:

test_prepared['Class Index'] = test_prepared['Class Index'] - 1
test_prepared['Description'] = test_prepared['Title'] + ' sep ' +  test_prepared['Description']

cv = CountVectorizer(ngram_range=(1,2), min_df = 10)
Y_cv = cv.fit_transform(test_prepared['Description'].apply(lambda x: np.str_(x)))

y = test_prepared['Class Index'].to_numpy()
dataset = NamesDataset(Y_cv, y)



# Для сабмитов

In [ ]:
test_prepared = clean_description(test_data)
y_train = train_data['Class Index']
X = cv.fit_transform(train_df['Description'].apply(lambda x: np.str_(x)))
x_train, x_valid = X, cv.transform(test_prepared['Description'].apply(lambda x: np.str_(x)))


  0%|          | 0/7600 [00:00<?, ?it/s]<ipython-input-17-67a93c553fc0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Description'][i] = text
100%|██████████| 7600/7600 [01:45<00:00, 71.76it/s]


In [ ]:
mnb = MultinomialNB()
mnb.fit(x_train,y_train)
pred = mnb.predict(x_valid)
test_results = pd.DataFrame({'ID': [i for i in range(test_prepared.shape[0])], 'Class Index': pred})

In [ ]:
test_results['Class Index'].value_counts()

2    1960
3    1903
4    1886
1    1851
Name: Class Index, dtype: int64

In [ ]:
test_results.to_csv('test_res_mnb.csv', header=True, sep=',', index=False)

In [ ]:
#test_prepared.drop(columns = ['Unnamed: 0'],axis = 1, inplace=True)
test_prepared.head()

,ID,Title,Description
0,0,Fears for T N pension after talks,union represent worker turner newall disappoi...
1,1,The Race is On: Second Private Team Sets Launc...,space.com toronto canada second\team rocketeer...
2,2,Ky. Company Wins Grant to Study Peptides (AP),ap company found chemistry researcher universi...
3,3,Prediction Unit Helps Forecast Wildfires (AP),ap barely dawn mike fitzpatrick start shift bl...
4,4,Calif. Aims to Limit Farm-Related Smog (AP),ap southern california smog fight agency go em...
